### Импорт библиотеки

In [1]:
import torch
import torch.nn as nn

**Создаем модель Sequential и определяем все слои в один "заход"**

In [12]:
model = nn.Sequential(
 nn.Linear(764, 100, device="cuda:0"),
 nn.ReLU(),
 nn.Linear(100, 50),
 nn.ReLU(),
 nn.Linear(50, 10),
 nn.Sigmoid()
)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

**Исследование модели**

In [3]:
print(model)

Sequential(
  (0): Linear(in_features=764, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=10, bias=True)
  (5): Sigmoid()
)


**Весь объект модели сохраняем в файле pickle.**

In [4]:
torch.save(model, "my_model.pickle")

**Извлечение модели**

In [5]:
torch.save(model.state_dict(), "my_model.pickle")

### Имплементация модели LSTM в PyTorch

In [6]:
import torch
import torch.nn as nn
class LSTMModel(nn.Module):  
    # input_d - количество ожидаемых признаков на входе. hidden_d - количество признаков в скрытом состоянии.
    def __init__(self, input_d, hidden_d, layer_d, output_d): 

        super(LSTMModel, self).__init__()

        # Определяем количество скрытых измерений и слоев
        self.hidden_dim = hidden_d
        self.layer_dim = layer_d

        # LSTM model
        self.lstm = nn.LSTM(input_d, hidden_d, layer_d, batch_first=True)

        # Определяем слой считывания с помощью функции fc().
        self.fc = nn.Linear(hidden_d, output_d)

    # Определяем функцию forward для создания прямого прохода для модели LSTM
    def forward(self, x):

        # Инициализируем скрытое состояние нулями при помощи zeros.
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Инициализируем состояние ячеек нулями при помощи zeros.
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Этот шаг выполняется тридцать раз. Мы отсоединяемся (detach), когда усекаем (truncate)
        # Backpropagation through time (BPTT - Градиентная методика обучения некоторых типов RNN). Если мы
        # не производим truncate, то вернемся к началу.
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out

# Объявляются переменные и создается объект модели LSTM.
input_dim = 30
hidden_dim = 120
output_dim = 15
layer_dim = 1
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)


**После инстанцирования модели мы инстанцируем потери (instantiation), вычисляя кросс-энтропийные
потери. Следующий фрагмент кода демонстрирует это:**

In [7]:
error = nn.CrossEntropyLoss()

**Далее мы инстанцируем оптимизатор с помощью SGD-оптимизатора:**

In [8]:
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)